In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#ÌïÑÏöî Ìå®ÌÇ§ÏßÄ ÏÑ§Ïπò
!pip install numpy==1.23.1
!pip install mxnet
!pip install gluonnlp
!pip install pandas tqdm
!pip install sentencepiece==0.1.96
!pip install transformers==4.8.2
!pip install torch==1.8.1

  Using cached mxnet-1.9.1-py3-none-manylinux2014_x86_64.whl (49.1 MB)
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.3
    Uninstalling graphviz-0.20.3:
      Successfully uninstalled graphviz-0.20.3
  Using cached gluonnlp-0.10.0.tar.gz (344 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp310-cp310-linux_x86_64.whl size=661751 sha256=5ff048b9897726a78a285def5ebb6fa7ebacce5456e153df64d5001990cad8ec
  Stored in directory: /root/.cache/pip/wheels/1a/1e/0d/99f55911d90f2b95b9f7c176d5813ef3622894a4b30fde6bd3
Successfully built gluonnlp
     ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 1.2/1.2 MB 7.1 MB/s eta 0:00:00
  Attempting uninstall: sentencepiece
    Found existing installation: sentencepiece 0.1.99
    Uninstalling sentencepiece-0.1.99:
      Successfully uninstalled sentencepiece-0.1.99
     ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚î

In [3]:
#KoBERT ÍπÉÌóàÎ∏åÏóêÏÑú Î∂àÎü¨Ïò§Í∏∞
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-6xtot8lj/kobert-tokenizer_9d7fd9824163440d8b049ce0780876bb
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-6xtot8lj/kobert-tokenizer_9d7fd9824163440d8b049ce0780876bb
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4633 sha256=5cc69f0a5cc45b07e3e4fe2d732b9707f70e59d0b3a8909196ece3700fd3eb4f
  Stored in directory: /tmp/pip-ephem-wheel-cache-ey6doftk/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-nmk6q259
  Running command git clone --filter=blob:none --quiet 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-nmk6q

In [4]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel
import pandas as pd

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
device = torch.device("cuda:0")
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/244 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


config.json:   0%|          | 0.00/535 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/369M [00:00<?, ?B/s]

In [6]:
# [AI Hub] Í∞êÏ†ï Î∂ÑÎ•òÎ•º ÏúÑÌïú ÎåÄÌôî ÏùåÏÑ± Îç∞Ïù¥ÌÑ∞ÏÖã
chatbot_data  = pd.read_csv("/content/drive/MyDrive/dataset/5·Ñé·Ö°·ÑÇ·Öß·Ü´·ÑÉ·Ö©_2·Ñé·Ö°.csv", encoding='cp949')

In [9]:
# "ÏÉÅÌô©" ÏÇ¨Ïö©ÌïòÍ∏∞
chatbot_data.loc[(chatbot_data['ÏÉÅÌô©'] == "fear"), 'ÏÉÅÌô©'] = 0  #Í≥µÌè¨ => 0
chatbot_data.loc[(chatbot_data['ÏÉÅÌô©'] == "surprise"), 'ÏÉÅÌô©'] = 1  #ÎÜÄÎûå => 1
chatbot_data.loc[(chatbot_data['ÏÉÅÌô©'] == "angry"), 'ÏÉÅÌô©'] = 2  #Î∂ÑÎÖ∏ => 2
chatbot_data.loc[(chatbot_data['ÏÉÅÌô©'] == "sadness"), 'ÏÉÅÌô©'] = 3  #Ïä¨Ìîî => 3
chatbot_data.loc[(chatbot_data['ÏÉÅÌô©'] == "neutral"), 'ÏÉÅÌô©'] = 4  #Ï§ëÎ¶Ω => 4
chatbot_data.loc[(chatbot_data['ÏÉÅÌô©'] == "happiness"), 'ÏÉÅÌô©'] = 5  #ÌñâÎ≥µ => 5
chatbot_data.loc[(chatbot_data['ÏÉÅÌô©'] == "disgust"), 'ÏÉÅÌô©'] = 6  #ÌòêÏò§ => 6

In [7]:
# 7Í∞úÏùò Í∞êÏ†ï class ‚Üí Ïà´Ïûê
chatbot_data.loc[(chatbot_data['emotion'] == "fear"), 'emotion'] = 0  #Í≥µÌè¨ => 0
chatbot_data.loc[(chatbot_data['emotion'] == "surprise"), 'emotion'] = 1  #ÎÜÄÎûå => 1
chatbot_data.loc[(chatbot_data['emotion'] == "angry"), 'emotion'] = 2  #Î∂ÑÎÖ∏ => 2
chatbot_data.loc[(chatbot_data['emotion'] == "sadness"), 'emotion'] = 3  #Ïä¨Ìîî => 3
chatbot_data.loc[(chatbot_data['emotion'] == "neutral"), 'emotion'] = 4  #Ï§ëÎ¶Ω => 4
chatbot_data.loc[(chatbot_data['emotion'] == "happiness"), 'emotion'] = 5  #ÌñâÎ≥µ => 5
chatbot_data.loc[(chatbot_data['emotion'] == "disgust"), 'emotion'] = 6  #ÌòêÏò§ => 6

KeyError: 'emotion'

In [11]:

# chatbot_data.sample(n=10)
# # 7Í∞úÏùò Í∞êÏ†ï class ‚Üí Ïà´Ïûê
# chatbot_data.loc[(chatbot_data['Unnamed: 2'] == "Í≥µÌè¨"), 'Unnamed: 2'] = 0  #Í≥µÌè¨ => 0
# chatbot_data.loc[(chatbot_data['Unnamed: 2'] == "ÎÜÄÎûå"), 'Unnamed: 2'] = 1  #ÎÜÄÎûå => 1
# chatbot_data.loc[(chatbot_data['Unnamed: 2'] == "Î∂ÑÎÖ∏"), 'Unnamed: 2'] = 2  #Î∂ÑÎÖ∏ => 2
# chatbot_data.loc[(chatbot_data['Unnamed: 2'] == "Ïä¨Ìîî"), 'Unnamed: 2'] = 3  #Ïä¨Ìîî => 3
# chatbot_data.loc[(chatbot_data['Unnamed: 2'] == "Ï§ëÎ¶Ω"), 'Unnamed: 2'] = 4  #Ï§ëÎ¶Ω => 4
# chatbot_data.loc[(chatbot_data['Unnamed: 2'] == "ÌñâÎ≥µ"), 'Unnamed: 2'] = 5  #ÌñâÎ≥µ => 5
# chatbot_data.loc[(chatbot_data['Unnamed: 2'] == "ÌòêÏò§"), 'Unnamed: 2'] = 6  #ÌòêÏò§ => 6

data_list = []
for q, label in zip(chatbot_data['Î∞úÌôîÎ¨∏'], chatbot_data['ÏÉÅÌô©'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

print(data)
print(data_list[:10])

# chatbot_data.sample(n=10)
# # 7Í∞úÏùò Í∞êÏ†ï class ‚Üí Ïà´Ïûê
# chatbot_data.loc[(chatbot_data['Emotion'] == "Í≥µÌè¨"), 'Emotion'] = 0  #Í≥µÌè¨ => 0
# chatbot_data.loc[(chatbot_data['Emotion'] == "ÎÜÄÎûå"), 'Emotion'] = 1  #ÎÜÄÎûå => 1
# chatbot_data.loc[(chatbot_data['Emotion'] == "Î∂ÑÎÖ∏"), 'Emotion'] = 2  #Î∂ÑÎÖ∏ => 2
# chatbot_data.loc[(chatbot_data['Emotion'] == "Ïä¨Ìîî"), 'Emotion'] = 3  #Ïä¨Ìîî => 3
# chatbot_data.loc[(chatbot_data['Emotion'] == "Ï§ëÎ¶Ω"), 'Emotion'] = 4  #Ï§ëÎ¶Ω => 4
# chatbot_data.loc[(chatbot_data['Emotion'] == "ÌñâÎ≥µ"), 'Emotion'] = 5  #ÌñâÎ≥µ => 5
# chatbot_data.loc[(chatbot_data['Emotion'] == "ÌòêÏò§"), 'Emotion'] = 6  #ÌòêÏò§ => 6

# data_list = []
# for q, label in zip(chatbot_data['Sentence'], chatbot_data['Emotion'])  :
#     data = []
#     data.append(q)
#     data.append(str(label))

#     data_list.append(data)

# print(data)
# print(data_list[:10])

['Í±±Ï†ïÌï¥Ï§òÏÑú Í≥†ÎßàÏõå.', '6']
[['Ìóê! ÎÇò Ïù¥Î≤§Ìä∏Ïóê ÎãπÏ≤® ÎêêÏñ¥.', '5'], ['ÎÇ¥Í∞Ä Ï¢ãÏïÑÌïòÎäî Ïù∏ÌîåÎ£®Ïñ∏ÏÑúÍ∞Ä Ïù¥Î≤§Ìä∏Î•º ÌïòÎçîÎùºÍ≥†. Í∑∏ÎûòÏÑú Í∑∏ÎÉ• Ïã†Ï≤≠ ÌïúÎ≤à Ìï¥Î¥§ÏßÄ.', '5'], ['Ìïú Î™Ö ÎΩëÎäî Í±∞ÏòÄÎäîÎç∞, Í∑∏Í≤å Î∞îÎ°ú ÎÇ¥Í∞Ä Îêú Í±∞Ïïº.', '5'], ['ÎãπÏó∞Ìûà ÎßàÏùåÏóê ÎìúÎäî ÏÑ†Î¨ºÏù¥ÎãàÍπê, Ïù¥Î≤§Ìä∏Ïóê ÎÇ¥Í∞Ä Ïã†Ï≤≠ ÌïúÎ≤à Ìï¥Î≥∏ Í±∞ÏßÄ. ÎπÑÏãº Í±∞Ïïº. Í∑∏ÎûòÏÑú Î™ª ÏÇ∞ Ìñ•ÏàòÏïº.', '5'], ['ÏóêÌîºÌÉÄÏù¥Ï†Ä Ï†ïÎßê Ï¢ãÏïÑÌï¥. Í∑∏ Í≤ÉÎèÑ Í¥úÏ∞ÆÏùÄ ÏÉùÍ∞ÅÏù∏ Í≤É Í∞ôÏï†.', '4'], ['ÎÇú Î∂ÄÌéò ÌòïÏãùÏùò ÏùåÏãùÎì§ÎèÑ Ï†ïÎßê Ï¢ãÏïÑÌï¥. Í∑∏ Í≤ÉÎèÑ Ï¢Ä ÏïåÎ†§ Ï§ò.', '4'], ['Ïùë. ÏôÑÏ†ÑÌûà ÎÅùÎÇú Í±∞Ïïº. Ìïú Îã¨ ÎèôÏïà Ï£ºÎßêÏóê Ïâ¨ÏßÄÎèÑ Î™ªÌïòÍ≥† ÏùºÎßå ÌñàÍ±∞Îì†.', '5'], ['Ïã†ÎÇòÎäî ÏùåÏïÖ Îì£Îäî Í≤ÉÎèÑ Ï¢ãÍ≥†, Ïñ¥Îîî Ïó¨Ìñâ Í∞ÄÍ≥† Ïã∂Í≥† Ïù¥ Í≤É Ï†Ä Í≤É Îã§ Ìï¥Î≥¥Í≥† Ïã∂Ïñ¥.', '5'], ['ÏπúÍµ¨Îì§ÎèÑ ÎÇ¥ Ïó∞ÎùΩ Í∏∞Îã§Î¶¨Í≥† ÏûàÏùÑ ÌÖêÎç∞ ÎÇ¥Í∞Ä ÍπåÎ®πÍ≥† ÏûàÏóàÎÑ§?', '5'], ['Í∑∏Îûò. ÏùºÎã®ÏùÄ ÏπúÍµ¨Îì§Î∂ÄÌÑ∞ ÎßåÎÇòÏÑú Ïó¨Ìñâ Í≥ÑÌöçÏóê ÎåÄÌï¥ÏÑú ÏñòÍ∏∞ Ï¢Ä Ìï¥Î¥êÏïº Îê

In [12]:
#train & test Îç∞Ïù¥ÌÑ∞Î°ú ÎÇòÎàÑÍ∏∞
from sklearn.model_selection import train_test_split
dataset_train, dataset_test = train_test_split(data_list, test_size = 0.2, shuffle = True, random_state = 32)


In [13]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower = False)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [14]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):

        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, vocab=vocab, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))


    def __len__(self):
        return (len(self.labels))

In [15]:
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [18]:
tok=tokenizer.tokenize

data_train = BERTDataset(dataset_train, 0, 1, tok, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test,0, 1, tok, vocab,  max_len, True, False)

train_dataloader = torch.utils.data.DataLoader(data_train, batch_size = batch_size, num_workers = 5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size = batch_size, num_workers = 5)

In [19]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 7,
                 dr_rate = None,
                 params = None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p = dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict = False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [20]:
# BERT  Î™®Îç∏ Î∂àÎü¨Ïò§Í∏∞
model = BERTClassifier(bertmodel,  dr_rate = 0.5).to(device)

# optimizerÏôÄ schedule ÏÑ§Ï†ï
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [21]:
optimizer = AdamW(optimizer_grouped_parameters, lr = learning_rate)
loss_fn = nn.CrossEntropyLoss() # Îã§Ï§ëÎ∂ÑÎ•òÎ•º ÏúÑÌïú loss function

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps = warmup_step, num_training_steps = t_total)

# calc_accuracy : Ï†ïÌôïÎèÑ Ï∏°Ï†ïÏùÑ ÏúÑÌïú Ìï®Ïàò
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

train_dataloader

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [22]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

<ipython-input-22-c43874aa7363>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/243 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


epoch 1 batch id 1 loss 2.022535562515259 train acc 0.140625
epoch 1 batch id 201 loss 0.5260992646217346 train acc 0.6075870646766169


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


...

epoch 4 test acc 0.9243559718969555


  0%|          | 0/243 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.013563944958150387 train acc 1.0
epoch 5 batch id 201 loss 0.07949571311473846 train acc 0.9780783582089553
epoch 5 train acc 0.9774948559670782


  0%|          | 0/61 [00:00<?, ?it/s]

epoch 5 test acc 0.9233313817330211


In [23]:
state = {
    'state_dict': model.state_dict(),
}

torch.save(state, "/content/drive/MyDrive/dataset/bert_classifier_model_v2.pth")

In [29]:
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)

    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)

        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("Í≥µÌè¨")
            elif np.argmax(logits) == 1:
                test_eval.append("ÎÜÄÎûå")
            elif np.argmax(logits) == 2:
                test_eval.append("Î∂ÑÎÖ∏")
            elif np.argmax(logits) == 3:
                test_eval.append("Ïä¨Ìîî")
            elif np.argmax(logits) == 4:
                test_eval.append("Ï§ëÎ¶Ω")
            elif np.argmax(logits) == 5:
                test_eval.append("ÌñâÎ≥µ")
            elif np.argmax(logits) == 6:
                test_eval.append("ÌòêÏò§")

        print(test_eval[0], "???")

In [30]:
inf_data  = pd.read_excel("/content/drive/MyDrive/dataset/tesla.xlsx")
inf_data = inf_data.drop(0)

In [31]:
inf_data = inf_data[["text", "emotion"]]
inf_data

text emotion
1           Ïò§Îäò ÌïòÎ£®ÎèÑ ÌûòÏ∞¨ ÌïòÎ£®ÎêòÏÑ∏Ïöî. Ïö∞Î¶¨Í∞Ä ÌÖåÏä¨Îùº ÎØøÏñ¥Ï£ºÏßÄ ÎàÑÍ∞Ä ÎØøÏñ¥Ï§çÎãàÍπå? „Öã      Í∏∞ÏÅ®
2   ÌÖåÏä¨ÎùºÏùò Ï£ºÍ∞ÄÍ∞Ä Ï≠âÏ≠â Îñ®Ïñ¥ÏßÄÏûê ÌÖåÏä¨ÎùºÏóê ÎåÄÌïú Ï†ïÎ≥¥Î•º Îã§Î£®Îäî Ïú†ÌäúÎ∏å, ‚ÄòÏò§ÎäòÏùò ÌÖåÏä¨Îùº...      Í∏∞ÏÅ®
3   ÏïÑÎÇ¥Í∞Ä ÎèôÎÑ§ ÏïÑÏ£ºÎ®∏ÎãàÎì§ ÌïúÎ∂ÑÏù¥ ÏÉùÏùºÏù¥Îùº Ï†êÏã¨ Î™®ÏûÑ Îã§ÎÖÄÏôîÎäîÎç∞, ÌôîÏ†ú Ï§ë ÏùºÎ∂ÄÍ∞Ä FS...      Í∏∞ÏÅ®
4   ÌÖåÏä¨ÎùºÎäî ÌòÑÏû¨ ÏÇ¨Ïã§ÏÉÅ FSDÏôÄ Î°úÎ≥¥ÌÉùÏãú Ïò¨Ïù∏ Î™®ÎìúÍ≥†, Îç∞Ïù¥ÌÑ∞ÏÑºÌÑ∞ Íµ¨Ï∂ïÏóê ÏóÑÏ≤≠ÎÇú ÎèàÏùÑ...      Í≥µÌè¨
5   ÌÖåÏä¨Îùº Ìï¥Í≥† Ïù∏Ïõê Ìï¥Í≥† ÏÇ¨Ïã§ Î™∞Îùº Ï∂úÍ∑º ÌõÑ ÏÇ¨ÏõêÏ¶ù ÌÜµÌïòÏßÄ ÏïäÏùÄ ÏÇ¨Ïã§ ÏïåÍ≥† ÏßëÏóê ÎèåÏïÑ...      ÌòêÏò§
..                                                ...     ...
66                       ÌÖåÏä¨Îùº Î™®Îç∏2. Í∞ÄÎßåÌûà Î≥¥Îãà ÏïÑÏù¥Ìè∞Ïùò seÎäêÎÇåÎÇòÎÑ§Ïöî      Ï§ëÎ¶Ω
67               Î®∏Ïä§ÌÅ¨Í∞Ä Ìä∏ÏúÑÌÑ∞Ïóê ÏãúÍ∞ÑÏùÑ ÎßéÏù¥ Ïì∞Îãà ÌÖåÏä¨Îùº ÌïòÎùΩÏóê ÌíÄÎ≤†ÌåÖÏùÑ ÌïòÏã¨Ïù¥      ÌòêÏò§
68  ÌòÑÎ™ÖÌïú ÏôÄÏù¥ÌîÑÎ•º ÎëêÏÖ®ÎÑ§Ïöî. ÌòÑÎ™ÖÌïú ÌïòÏù¥ÌîÑÎ•º Í≥†Î•∏ ÏïàÎ™©Ï≤òÎüº ÌÖåÏä¨Îùº ÎòêÌïú Ïã§ÎßùÏãúÌÇ§ÏßÄ Ïïä...      Í∏∞ÏÅ®
69  Ï°∞Í∏àÎßå Ï∞∏ÏúºÏÑ∏Ïöî $tsla Ï£ºÏ£ºÎãòÎì§~ ÏùºÎ°†Î®∏Ïä§ÌÅ¨ ÌòïÏù¥ ÏûàÏûñÏïÑÏöî~ ÌÖåÏä¨Îùº Í∞úÏù∏Ï£ºÏ£ºÎì§Ïù¥...      Í∏∞ÏÅ®
70                   ÌÖåÏä¨Îùº Ïä§ÌÜ†Ïñ¥Ïóê Í∞ÄÏãúÎ©¥ Ïò§ÌÜ†ÌååÏùºÎüø Ïª¥Ìì®ÌÑ∞Î•º Î≥º Ïàò ÏûàÏäµÎãàÎã§      Ï§ëÎ¶Ω

[70 rows x 2 columns]

In [32]:
for index, row in inf_data.iterrows():
    text = row['text']
    emotion = row['emotion']
    predict(text)
    print(text)
    print(emotion)

    print("-------------------")

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Ïä¨Ìîî ???
Ïò§Îäò ÌïòÎ£®ÎèÑ ÌûòÏ∞¨ ÌïòÎ£®ÎêòÏÑ∏Ïöî. Ïö∞Î¶¨Í∞Ä ÌÖåÏä¨Îùº ÎØøÏñ¥Ï£ºÏßÄ ÎàÑÍ∞Ä ÎØøÏñ¥Ï§çÎãàÍπå? „Öã
Í∏∞ÏÅ®
-------------------
Ïä¨Ìîî ???
ÌÖåÏä¨ÎùºÏùò Ï£ºÍ∞ÄÍ∞Ä Ï≠âÏ≠â Îñ®Ïñ¥ÏßÄÏûê ÌÖåÏä¨ÎùºÏóê ÎåÄÌïú Ï†ïÎ≥¥Î•º Îã§Î£®Îäî Ïú†ÌäúÎ∏å, ‚ÄòÏò§ÎäòÏùò ÌÖåÏä¨Îùº Îâ¥Ïä§‚ÄôÏóê Ï°∞Î°±Ïùò ÎåìÍ∏ÄÎì§Ïù¥ ÎßéÏù¥ Î≥¥Ïù¥Í∏∞ ÏãúÏûëÌñàÎã§. ÌÖåÏä¨ÎùºÏóê ÎåÄÌïú Ï°∞Î°±, ÎπÑÎÇúÍ∏ÄÏù¥ Ï†êÏ∞® ÎäòÏñ¥ÎÇòÎã§Í∞Ä Ïñ¥Îäê ÏàúÍ∞Ñ Î¨¥Í¥ÄÏã¨Ïóê Í∑∏Ïπ† ÎïåÍ∞Ä Î∞îÎã•Ïù¥ÏóàÍ≥† Ï†êÏ†ê Í∑∏ÎïåÍ∞Ä Îã§Í∞ÄÏò§Í≥† ÏûàÏùåÏùÑ Ïïå Ïàò ÏûàÎã§. Î∞îÎã•ÏùÑ Ïû°Îäî Í≤ÉÏùÄ Ïã†Ïùò‚Ä¶
Í∏∞ÏÅ®
-------------------
ÌñâÎ≥µ ???
ÏïÑÎÇ¥Í∞Ä ÎèôÎÑ§ ÏïÑÏ£ºÎ®∏ÎãàÎì§ ÌïúÎ∂ÑÏù¥ ÏÉùÏùºÏù¥Îùº Ï†êÏã¨ Î™®ÏûÑ Îã§ÎÖÄÏôîÎäîÎç∞, ÌôîÏ†ú Ï§ë ÏùºÎ∂ÄÍ∞Ä FSD. ÎåÄÎ∂ÄÎ∂Ñ ÌÖåÏä¨Îùº Î™®ÏãúÍ±∞ÎÇò Î™∞ÏïòÏóàÍ∏∞ ÎïåÎ¨∏Ïóê Ïù¥ÏïºÍ∏∞Í∞Ä Ïûò ÌÜµÌïòÎäî ÎìØ. 
Í∏∞ÏÅ®
-------------------
Ïä¨Ìîî ???
ÌÖåÏä¨ÎùºÎäî ÌòÑÏû¨ ÏÇ¨Ïã§ÏÉÅ FSDÏôÄ Î°úÎ≥¥ÌÉùÏãú Ïò¨Ïù∏ Î™®ÎìúÍ≥†, Îç∞Ïù¥ÌÑ∞ÏÑºÌÑ∞ Íµ¨Ï∂ïÏóê ÏóÑÏ≤≠ÎÇú ÎèàÏùÑ Ïì∞Í≥† ÏûàÏùå. Îç∞Ïù¥ÌÑ∞ÏÑºÌÑ∞ Íµ¨Ï∂ï ÎπÑÏö©ÏùÑ ÌïúÍ∫ºÎ≤àÏóê ÎßéÏ